In [9]:
import os, tarfile
import re
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
input_path = "/data/projects/Tissue_specific_data/TissueEnhanced/DNABERT_Finetuned_Data/brain/1_1/"

In [11]:
df = pd.read_csv(input_path+"Brain-Hsk_test_1-1_data_All_30.csv", sep=",")

In [12]:
df

,coord,sequence,A_Fraction,C_Fraction,G_Fraction,T_Fraction,PurPyr_Fraction,AmKe_Fraction,WeSt_Fraction,CpG1,...,eGFP-ZSCAN23-HEK293_0,eGFP-ZSCAN26-HEK293_0,eGFP-ZSCAN30-HEK293_0,eGFP-ZSCAN4-HEK293_0,eGFP-ZSCAN5A-HEK293_0,eGFP-ZSCAN5C-HEK293_0,eGFP-ZXDB-HEK293_0,tissue,trans_type,label
0,chr3:129439449-129441948,ATGACTTCCTAGGATCCTCCAACCCCAAGGGGACCTCTTTCACAAA...,0.2308,0.2596,0.2560,0.2536,-0.0264,-0.0192,-0.0312,0.231293,...,1,0,1,2,1,0,2,all,Hsk,0.0
1,chrX:153515676-153518175,GCCTCCCGGCCAGGGGCCAAAGTCCAGCTTACAACCTGCTCCGGGT...,0.1548,0.3044,0.3372,0.2036,-0.0160,-0.0816,-0.2832,0.329732,...,0,0,0,1,0,0,0,brain,Tsp,1.0
2,chr17:46978754-46981253,AAGACGCCGAAGACCACGGTAAGCGACAGCACGCAGAGCACGGCGA...,0.2156,0.2908,0.2544,0.2392,-0.0600,0.0128,-0.0904,0.268107,...,0,0,0,0,0,0,0,brain,Tsp,1.0
3,chr1:21619542-21622041,CCCACCTCCCATGCATACCTATGTGAAACGCAGACGGGGCTGTGGG...,0.2492,0.3288,0.2540,0.1680,0.0064,0.1560,-0.1656,0.179272,...,0,0,0,0,0,0,0,brain,Tsp,1.0
4,chr3:10533634-10536133,GGGTAGAAGAATGAAGCAGGACCAGGTGAGGTGGCTGGCTTTTCAT...,0.2308,0.2344,0.2304,0.3044,-0.0776,-0.0696,0.0704,0.111244,...,0,0,0,1,0,0,0,brain,Tsp,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,chr12:21394938-21397437,GCAGGCACTTCACAAAGCACCGTAAGTGTGCTGTGAAATAACCAAG...,0.3592,0.2272,0.2396,0.1740,0.1976,0.1728,0.0664,0.152861,...,0,0,0,0,0,0,0,brain,Tsp,1.0
355,chr12:130870082-130872581,TGCAGGTAAAGTTAAAGTCTTCTTTGACTCCTCTTCTCAGTTCCAT...,0.1740,0.3164,0.2956,0.2140,-0.0608,-0.0192,-0.2240,0.367347,...,1,0,1,0,0,0,1,all,Hsk,0.0
356,chr1:2071986-2074485,ATCCTTTTCATTTTATTCAGCCAATTAAAAGCGTAGAAATCCTAGC...,0.1632,0.2684,0.3480,0.2204,0.0224,-0.1368,-0.2328,0.284914,...,0,0,0,0,0,0,0,brain,Tsp,1.0
357,chr7:26199745-26202244,GTTACTCAAAGGAAAAAAAAGACAGGAAAAACATAAAATGGTTTCT...,0.1888,0.3504,0.2864,0.1744,-0.0496,0.0784,-0.2736,0.389756,...,0,0,2,0,0,0,1,all,Hsk,0.0


In [13]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [14]:
df_out= pd.DataFrame()
df_out['Sequence'] = df['sequence'].apply(seq2kmer, args=(6,))
df_out["Label"] = df['label'].astype('int')

In [15]:
df_out

,Sequence,Label
0,ATGACT TGACTT GACTTC ACTTCC CTTCCT TTCCTA TCCT...,0
1,GCCTCC CCTCCC CTCCCG TCCCGG CCCGGC CCGGCC CGGC...,1
2,AAGACG AGACGC GACGCC ACGCCG CGCCGA GCCGAA CCGA...,1
3,CCCACC CCACCT CACCTC ACCTCC CCTCCC CTCCCA TCCC...,1
4,GGGTAG GGTAGA GTAGAA TAGAAG AGAAGA GAAGAA AAGA...,1
...,...,...
354,GCAGGC CAGGCA AGGCAC GGCACT GCACTT CACTTC ACTT...,1
355,TGCAGG GCAGGT CAGGTA AGGTAA GGTAAA GTAAAG TAAA...,0
356,ATCCTT TCCTTT CCTTTT CTTTTC TTTTCA TTTCAT TTCA...,1
357,GTTACT TTACTC TACTCA ACTCAA CTCAAA TCAAAG CAAA...,0


In [16]:
df_out.to_csv(input_path+"/dev.tsv", sep="\t", index=False)